In [ ]:
import pandas as pd
import glob
import os

# Caminhos
path_fbref = r"d:/Diogo/Ambiente de Trabalho/PROJETO/Datasets/FBRef/dataset_completo_final.csv"
path_fd = r"d:/Diogo/Ambiente de Trabalho/PROJETO/Datasets/football_data_co_uk/"

# Carregar FBRef
df_fbref = pd.read_csv(path_fbref)

# Carregar Jogos
files = glob.glob(os.path.join(path_fd, "P1_*.csv"))
dfs = [pd.read_csv(f) for f in files]
df_matches = pd.concat(dfs, ignore_index=True)

# Tratar datas e remover lixo
df_matches['Date'] = pd.to_datetime(df_matches['Date'], dayfirst=True, errors='coerce')
df_matches = df_matches.dropna(subset=['Date']).copy().sort_values('Date')
# Remover colunas Unnamed
df_matches = df_matches.loc[:, ~df_matches.columns.str.contains('^Unnamed')]

print("Dados carregados e limpos.")

In [ ]:
# Mapeamento de nomes
mapping = {
    "AVS": "AVS Futebol", "Academica": "Académica", "Beira Mar": "Beira-Mar",
    "Famalicao": "Famalicão", "Maritimo": "Marítimo", "Pacos Ferreira": "Paços de Ferreira",
    "Sp Braga": "Braga", "Sp Lisbon": "Sporting CP", "Setubal": "Vitória Setúbal",
    "Uniao Madeira": "União", "Guimaraes": "Vitória", "Gil Vicente": "Gil Vicente",
    "Nacional": "Nacional", "Estrela": "Estrela", "Leiria": "União de Leiria",
    "Leixoes": "Leixões", "Feirense": "Feirense", "Belenenses": "Belenenses"
}

df_matches['HomeTeam'] = df_matches['HomeTeam'].str.strip().replace(mapping)
df_matches['AwayTeam'] = df_matches['AwayTeam'].str.strip().replace(mapping)

df_fbref['Equipa'] = df_fbref['Equipa'].replace({
    "Gil Vicente FC": "Gil Vicente", "B-SAD": "Belenenses", "Sporting": "Sporting CP"
})

print("Nomes corrigidos.")

In [ ]:
# Calcular épocas
df_matches['Season'] = df_matches['Date'].apply(lambda x: f"{x.year}-{x.year+1}" if x.month >= 8 else f"{x.year-1}-{x.year}")
df_matches['Previous_Season'] = df_matches['Season'].apply(lambda x: f"{int(x.split('-')[0])-1}-{int(x.split('-')[0])}")

# Preparar dados históricos (FBRef)
cols_renomeadas = {c: f"hist_{c}" for c in df_fbref.columns if c not in ['Equipa', 'Epoca']}
df_hist = df_fbref.rename(columns=cols_renomeadas)

# --- IMPUTAÇÃO: Calcular média das 3 piores (Relegation Zone) ---
relegation_stats = []
for season in df_hist['Epoca'].unique():
    # Ordenar por Pontos (assumindo que existe 'hist_Pontos')
    season_data = df_hist[df_hist['Epoca'] == season].sort_values('hist_Pontos')
    if not season_data.empty:
        bottom_3 = season_data.head(3).mean(numeric_only=True)
        bottom_3['Epoca'] = season
        relegation_stats.append(bottom_3)

df_relegation = pd.DataFrame(relegation_stats)

# Merge Inicial
df_final = pd.merge(df_matches, df_hist, left_on=['HomeTeam', 'Previous_Season'], right_on=['Equipa', 'Epoca'], how='left')
df_final = df_final.rename(columns={c: f"Home_{c}" for c in df_hist.columns if 'hist_' in c})

df_final = pd.merge(df_final, df_hist, left_on=['AwayTeam', 'Previous_Season'], right_on=['Equipa', 'Epoca'], how='left', suffixes=('', '_Away'))
df_final = df_final.rename(columns={c: f"Away_{c}" for c in df_hist.columns if 'hist_' in c})

df_final = df_final.drop(columns=['Equipa', 'Epoca', 'Equipa_Away', 'Epoca_Away'], errors='ignore')

# Aplicar Imputação (Preencher NaNs com média da relegação)
for prefix in ['Home_', 'Away_']:
    # Criar DF de defaults para a época correspondente
    df_defaults = pd.merge(df_final[['Previous_Season']], df_relegation, left_on='Previous_Season', right_on='Epoca', how='left')
    rename_dict = {c: f"{prefix}{c}" for c in df_relegation.columns if c != 'Epoca'}
    df_defaults = df_defaults.rename(columns=rename_dict)
    
    for col in rename_dict.values():
        if col in df_final.columns:
            df_final[col] = df_final[col].fillna(df_defaults[col])

print(f"Merge e Imputação feitos. Shape: {df_final.shape}")

In [21]:
# --- RENOMEAR COLUNAS PARA PORTUGUÊS ---
col_map_pt = {
    'Date': 'Data', 'Time': 'Hora', 'HomeTeam': 'Equipa_Casa', 'AwayTeam': 'Equipa_Visitante',
    'FTHG': 'Golos_Casa_Final', 'FTAG': 'Golos_Visitante_Final', 'FTR': 'Resultado_Final',
    'HTHG': 'Golos_Casa_Intervalo', 'HTAG': 'Golos_Visitante_Intervalo', 'HTR': 'Resultado_Intervalo',
    'HS': 'Remates_Casa', 'AS': 'Remates_Visitante', 'HST': 'Remates_Alvo_Casa', 'AST': 'Remates_Alvo_Visitante',
    'HF': 'Faltas_Casa', 'AF': 'Faltas_Visitante', 'HC': 'Cantos_Casa', 'AC': 'Cantos_Visitante',
    'HY': 'Amarelos_Casa', 'AY': 'Amarelos_Visitante', 'HR': 'Vermelhos_Casa', 'AR': 'Vermelhos_Visitante',
    'B365H': 'Odd_Casa_Bet365', 'B365D': 'Odd_Empate_Bet365', 'B365A': 'Odd_Visitante_Bet365',
    'AvgH': 'Odd_Casa_Media', 'AvgD': 'Odd_Empate_Media', 'AvgA': 'Odd_Visitante_Media'
}

df_final = df_final.rename(columns=col_map_pt)

# Guardar
df_final.to_csv("dataset_final_merged.csv", index=False)
print("Ficheiro dataset_final_merged.csv criado com nomes em Português.")

Ficheiro dataset_final_merged.csv criado com nomes em Português.
